In [1]:
import numpy as np
import xarray
import pandas as pd

In [2]:
GFRAC = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_NEW.nc", engine="netcdf4")
GAREA = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")
GLCT = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")
gbiomass_selisih = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/gbiomass_biomass_sumdim_manip_selisih_AM1.nc", engine="netcdf4")

In [5]:
GLCT_1st = xarray.open_dataset('D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc', engine="netcdf4")

In [4]:
garea_2020 = GAREA.isel(time=slice(0, 4))
garea_ha = np.nan_to_num(np.multiply(garea_2020['GAREA'].to_numpy(), 100))
gfrac_2020 = GFRAC.isel(time=slice(0, 4))

1st claffication:
natveg_to_biof
natveg_to_grass
	
	
	
land abandonment:
['agri_to_natveg', 'agri_to_regfor', 'biof_to_natveg',
       'biof_to_regfor', 'grass_to_natveg', 'grass_to_regfor']

In [5]:
GLCT_nav2agri = np.zeros((3, 2160, 4320), dtype="<U14")

for n in range(3):
    GLCT_nav2agri[n] = np.where(GLCT_1st['GLCT_1st'].isel(time=n) == 'natveg_to_biof', GLCT['GLCT_trans'].isel(time=n), '')

## gantian

In [17]:
for i in range(3):
    GLCT_nav2agri[i] = np.where(
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_regfor')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'biof_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'biof_to_regfor')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_regfor'),
                                GLCT['GLCT_trans'].isel(time=n), '')

In [18]:
GLCT_nav2agrinetcdf = xarray.Dataset({
    "GLCT_nav2agri":(["time","latitude", "longitude"], GLCT_nav2agri)
},coords={
        "time": pd.date_range(start='1975-01-01', end='1985-01-01', freq='5YS'),
        "latitude": GFRAC.coords["latitude"].to_numpy(),
        "longitude": GFRAC.coords["longitude"].to_numpy()
})

In [7]:
GBIOMASS_selsisih_nav2agri = np.zeros((3, 2160, 4320), dtype='float32')

for n in range(3):
    GBIOMASS_selsisih_nav2agri[n] = np.where(GLCT_1st['GLCT_1st'].isel(time=n) == 'natveg_to_agri', gbiomass_selisih['gbiomass_manip_selisih'].isel(time=n), 0)

## gantian

In [19]:
for i in range(3):
    GBIOMASS_selsisih_nav2agri[i] = np.where(
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'agri_to_regfor')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'biof_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'biof_to_regfor')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_natveg')|
                                (GLCT_1st['GLCT_1st'].isel(time=i) == 'grass_to_regfor'),
                                gbiomass_selisih['gbiomass_manip_selisih'].isel(time=i), 0) #diganti dengan GBIOMAS sumdim manip 

In [12]:
# n+1 = 1970 ambil 1975
natveg2crops = np.zeros((17, 3, 2160, 4320), dtype="float32")

for i in range(17):
    for n in range(3):
        natveg2crops[i][n] = np.nan_to_num(GBIOMASS_selsisih_nav2agri[n]) #diganti GBIOMAS slisih 

In [14]:
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")
luh_static = xarray.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/gpw_v4_national_identifier_grid_rev11_5_min_finall.nc", engine="netcdf4")


ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((2160, 4320), dtype="<U32") #rubah ke <U64

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            
country = xarray.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": GFRAC.coords["longitude"].to_numpy(), "latitude": GFRAC.coords["latitude"].to_numpy()})

In [ ]:
gfrac_ngfbfc = [element.strip() for element in GFRAC.coords['NGFBFC'].data.astype('str').tolist()]

GBIOMASS_selsisih_nav2agri_netcdf = xarray.Dataset(
coords={
        "time": pd.date_range(start='1975-01-01', end='1985-01-01', freq='5YS'),
        "latitude": GFRAC.coords["latitude"].to_numpy(),
        "longitude": GFRAC.coords["longitude"].to_numpy(),
        "origin": GLCT_nav2agrinetcdf['GLCT_nav2agri'],
        "country": country['country']
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, GBIOMASS_selsisih_nav2agri[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
GBIOMASS_selsisih_nav2agri_netcdf = GBIOMASS_selsisih_nav2agri_netcdf.assign(data_vars)
GBIOMASS_selsisih_nav2agri_netcdf

In [ ]:
GBIOMASS_selsisih_nav2agri_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_selsisih_nav2agri_netcdf.nc", mode="w", format="NETCDF4")

In [20]:
# deforestasi_netcdf = xarray.Dataset({
#         "land abandonment":(["time", "latitude", "longitude"], GAREA_nav2agri)
#     },
#     coords={
#         "time": pd.date_range(start='1975-01-01', end='1985-01-01', freq='5YS'),
#         "latitude": GFRAC.coords["latitude"].to_numpy(), 
#         "longitude": GFRAC.coords["longitude"].to_numpy(),
#         "origin": GLCT_nav2agrinetcdf['GLCT_nav2agri'],
#         "country": country['country']
#     })
# deforestasi_netcdf

<xarray.Dataset>
Dimensions:           (time: 3, latitude: 2160, longitude: 4320)
Coordinates:
  * time              (time) datetime64[ns] 1975-01-01 1980-01-01 1985-01-01
  * latitude          (latitude) float64 89.96 89.88 89.79 ... -89.87 -89.96
  * longitude         (longitude) float64 -180.0 -179.9 -179.8 ... 179.9 180.0
    origin            (time, latitude, longitude) <U14 '' '' '' '' ... '' '' ''
    country           (latitude, longitude) <U32 'ocean' 'ocean' ... 'ocean'
Data variables:
    land abandonment  (time, latitude, longitude) float32 0.0 0.0 ... 0.0 0.0

In [21]:
# deforestasi_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/land abandonment.nc", mode="w", format="NETCDF4")

In [54]:
# gfrac_ngfbfc = [element.strip() for element in GFRAC.coords['NGFBFC'].data.astype('str').tolist()]

# natveg2cropsnetcdf = xarray.Dataset(
# coords={
#         "time": pd.date_range(start='1975-01-01', end='2020-01-01', freq='5YS'),
#         "latitude": GFRAC.coords["latitude"].to_numpy(),
#         "longitude": GFRAC.coords["longitude"].to_numpy(),
#         "origin": GLCT_nav2agrinetcdf['GLCT_nav2agri'],
#         "country": country['country']
#     })

# coords = ("time", "latitude", "longitude")
# data_vars = {
#     ngfbc: (coords, natveg2crops[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
# }
# natveg2cropsnetcdf = natveg2cropsnetcdf.assign(data_vars)
# natveg2cropsnetcdf

In [57]:
# natveg2cropsnetcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/natveg_to_crops3thn_country.nc", mode="w", format="NETCDF4")

In [107]:
ds = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/land abandonment.nc", engine="netcdf4")

In [17]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]
n = 1

ds.isel(time=slice(0,3)).sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

,latitude,longitude,origin,country,grass,Wheat,Rice,Maize,Tropical cereals,Other temperate cereals,...,Soybeans,Temperate oil crops,Tropical oil crops,Temperate roots & tubers,Tropical roots & tubers,Sugar crops,Oil & palm fruit,Vegetables & fruits,Other non-food & luxury & spices,Plant based fibres
time,,,,,,,,,,,,,,,,,,,,,
1975-01-01,-7.125,109.374999,,Indonesia,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1980-01-01,-7.125,109.374999,,Indonesia,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1985-01-01,-7.125,109.374999,trof,Indonesia,1540.127319,0.0,1153.216553,948.137695,0.0,0.0,...,691.122498,0.0,1375.648926,32.723808,917.123718,223.958801,0.013884,896.234253,521.584778,58.423744


In [61]:
# x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
# y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]
# n = 3

GLCT_1st['GLCT_1st'].isel(time=slice(0, 11)).isel(latitude=432, longitude=2994).values

array(['grass_to_grass', 'grass_to_grass', 'grass_to_grass',
       'grass_to_grass', 'grass_to_grass', 'grass_to_grass',
       'grass_to_grass', 'grass_to_grass', 'grass_to_grass',
       'grass_to_grass'], dtype='<U14')

In [63]:
np.where(GLCT['GLCT_trans'].isel(time=0).values == "natveg_to_grass")

(array([], dtype=int64), array([], dtype=int64))